In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pysindy as ps

# Define simple linear system: dx/dt = A x
A = np.array([[0, 1, 0],
              [0, 0, 1],
              [-1, -1, -1]])

def rhs(t, x):
    return A @ x

# Initial condition
x0 = [1, 0, 0]

# Time grid
t_span = (0, 10)
dt = 0.01
t_eval = np.arange(t_span[0], t_span[1], dt)

# Integrate system
sol = solve_ivp(rhs, t_span, x0, t_eval=t_eval, rtol=1e-10)

x = sol.y.T  # Shape: (T, 3)
t = sol.t    # Shape: (T,)

# Compute derivatives explicitly
x_dot = np.gradient(x, dt, axis=0)

# Confirm shapes
print("[DEBUG] x.shape     =", x.shape)
print("[DEBUG] x_dot.shape =", x_dot.shape)

# Define SINDy model (no diff method needed now)
optimizer = ps.SR3(threshold=0.05, nu=1e-2)
library = ps.PolynomialLibrary(degree=2)

model = ps.SINDy(
    optimizer=optimizer,
    feature_library=library,
    feature_names=["x", "y", "z"],
    discrete_time=False
)

# Fit using explicit derivative
model.fit(x, x_dot=x_dot, t=dt, quiet=True)
model.print()
model.coefficients()

[DEBUG] x.shape     = (1000, 3)
[DEBUG] x_dot.shape = (1000, 3)
(x)' = -5.893 1 + 1.000 y + 5.893 x^2 + 11.787 x y + 11.786 y^2 + 11.787 y z + 5.893 z^2
(y)' = 1.053 1 + 1.000 z + -1.053 x^2 + -2.106 x y + -2.106 y^2 + -2.106 y z + -1.053 z^2
(z)' = 45.792 1 + -0.998 x + -1.000 y + -0.998 z + -45.793 x^2 + -91.585 x y + -91.584 y^2 + -91.585 y z + -45.791 z^2


array([[ -5.89320894,   0.        ,   0.99997823,   0.        ,
          5.8932076 ,  11.7866753 ,   0.        ,  11.78647936,
         11.78668802,   5.89323845],
       [  1.05294402,   0.        ,   0.        ,   0.99997394,
         -1.05303607,  -2.10626413,   0.        ,  -2.1059157 ,
         -2.10626314,  -1.05285717],
       [ 45.7916576 ,  -0.99805823,  -0.9998961 ,  -0.99804298,
        -45.79316326, -91.58490997,   0.        , -91.58383807,
        -91.58485416, -45.79057335]])